# Notebook to extract LMC and SMC Cadence

- Last update April 1 st 2021


- http://astro-lsst-01.astro.washington.edu:8080/ (baseline included)
- http://astro-lsst-01.astro.washington.edu:8081/  (rolling cadences included)
- http://astro-lsst-01.astro.washington.edu:8082/


In [103]:
# Check the version of MAF - the output should be version 2.4.2 or higher.
# 'unknown' means you're running from a git repo version,
# and have made changes to your local code

import lsst.sims.maf
lsst.sims.maf.__version__

'2.13.0.sims-93-g8bc2eb33+a14e41332a'

In [104]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [105]:
# import our python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

from lsst.sims.maf.db import ResultsDb

In [106]:
import sys
print(sys.path)

['/Users/dagoret/MacOSX/GitHub/LSST/MyCadence/ExtractLMC', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_almanac/master-g021b69e146+616205b9df/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_downtimeModel/master-g55f72efa65+3e384ed16a/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_cloudModel/master-ge3724df529+632df0f48d/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_seeingModel/master-ga4bf72ea44+6e62bc95d5/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_featureScheduler/master-g5f42dc1c76+c0a10aa3f3/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_skybrightness_pre/2.13.0.sims-11-g9ab127b+

### Specify my other python path (my installation)

In [107]:
otherpythonpaths=['/Users/dagoret/anaconda3/lib/python37.zip', '/Users/dagoret/anaconda3/lib/python3.7', '/Users/dagoret/anaconda3/lib/python3.7/lib-dynload', '', '/Users/dagoret/.local/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/dagoret/.ipython']

In [108]:
for thepath in otherpythonpaths:
    sys.path.append(thepath) # go to parent dir

In [109]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Astronomy

In [110]:
gindex=0

In [111]:
Galaxy_Name=["lmc","smc"]

In [112]:
Galaxy_Delta_RA=[10.75,5.33]
Galaxy_Delta_DEC=[9.17,3.083]

In [113]:
GName=Galaxy_Name[gindex]
galaxy_delta_ra=Galaxy_Delta_RA[gindex]
galaxy_delta_dec=Galaxy_Delta_DEC[gindex]

## astroquery

In [114]:
import astroquery
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

In [115]:
from astroquery.simbad import Simbad
result_table = Simbad.query_object(GName)
result_table.pprint(show_unit=True)

MAIN_ID      RA        DEC    ... COO_QUAL COO_WAVELENGTH     COO_BIBCODE    
          "h:m:s"    "d:m:s"  ...                                            
-------- ---------- --------- ... -------- -------------- -------------------
NAME LMC 05 23 34.6 -69 45 22 ...        D              O 2003A&A...412...45P


In [116]:
Angle(result_table["RA"][0]+" hour")

<Angle 5.39294444 hourangle>

In [117]:
Angle(result_table["DEC"][0]+" degrees")

<Angle -69.75611111 deg>

In [118]:
galaxy_coord=SkyCoord(ra=result_table["RA"][0]+"hour",dec=result_table["DEC"][0]+ "degrees")
galaxy_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (80.89416667, -69.75611111)>

In [119]:
galaxy_ra=galaxy_coord.ra.deg
galaxy_dec=galaxy_coord.dec.deg

In [120]:
print(galaxy_ra,galaxy_dec)

80.89416666666666 -69.75611111111111


# Config

## Database selection

In [121]:
dbindex=3

In [122]:
filters = ['u', 'g', 'r', 'i', 'z', 'y']
colors = {'u': 'cyan', 'g': 'g', 'r': 'y','i': 'r', 'z': 'm', 'y': 'k'}
Nfilt=len(filters)

In [151]:
database=["baseline2018a.db",\
          "baseline_v1.5_10yrs.db",\
          "baseline_2snaps_v1.5_10yrs.db",\
          "rolling_nm_scale1.0_nslice2_v1.7_10yrs.db",\
          "rolling_nm_scale1.0_nslice3_v1.7_10yrs.db",\
          "rolling_scale0.2_nslice3_v1.7_10yrs.db"
         ]

In [124]:
dbfileName=database[dbindex]

In [125]:
dbtag=dbfileName.split(".")[0]

In [126]:
dbtag

'baseline_2snaps_v1'

# Prepare access to database

## SQL Selection

In [127]:
raCol = 'fieldRA'
decCol = 'fieldDec'

In [128]:
sqlWhere = \
    'fieldRA < %f and ' \
    'fieldRA > %f and ' \
    'fieldDEC < %f and ' \
    'fieldDEC > %f  ' \
     % (galaxy_ra+ galaxy_delta_ra/2., galaxy_ra- galaxy_delta_ra/2.,galaxy_dec+galaxy_delta_dec/2.,galaxy_dec-galaxy_delta_dec/2)

In [129]:
sqlWhere

'fieldRA < 86.269167 and fieldRA > 75.519167 and fieldDEC < -65.171111 and fieldDEC > -74.341111  '

## From Tristan

In [130]:
RAMIN = 70.6 
RAMAX = 91.2 
DECMIN = -73.26 
DECMAX =-66.26

In [131]:
sqlWhere = \
    'fieldRA < %f and ' \
    'fieldRA > %f and ' \
    'fieldDEC < %f and ' \
    'fieldDEC > %f  ' \
     % (RAMAX, RAMIN,DECMAX,DECMIN)

In [132]:
sqlWhere

'fieldRA < 91.200000 and fieldRA > 70.600000 and fieldDEC < -66.260000 and fieldDEC > -73.260000  '

## Metric

In [133]:
dbfields=['observationId','night','observationStartMJD','fiveSigmaDepth','filter','airmass','skyBrightness','seeingFwhmEff','fieldId','fieldRA','fieldDec']

In [134]:
metric=metrics.PassMetric(cols=dbfields)

##  Database to open

In [135]:
opsdb = db.OpsimDatabase('/Users/dagoret/DATA/Cadence/'+dbfileName)
outDir = 'output_galaxycadence'
resultsDb = db.ResultsDb(outDir=outDir)

## Slicer

In [136]:
slicer_uni=slicers.UniSlicer()
slicer_point = slicers.UserPointsSlicer(galaxy_ra, galaxy_dec, lonCol='fieldRA', latCol='fieldDec', latLonDeg='degrees')
nside = 512
slicer_heal = slicers.HealpixSlicer(nside=nside, lonCol=raCol, latCol=decCol)

Healpix slicer using NSIDE=512, approximate resolution 6.870973 arcminutes


# Run

## Bundle

- Bundles dict for all filter

In [137]:
bundles = {}
bundles[GName] = metricBundles.MetricBundle(metric, slicer_uni, sqlWhere)
print(bundles[GName].dbCols)

{'skyBrightness', 'filter', 'seeingFwhmEff', 'airmass', 'night', 'fieldId', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA', 'observationId', 'fieldDec'}


In [138]:
bgroup = metricBundles.MetricBundleGroup(bundles, opsdb, outDir=outDir,resultsDb=resultsDb)

In [139]:
bgroup.runAll()
bgroup.plotAll(closefigs=False)

Querying database SummaryAllProps with constraint fieldRA < 91.200000 and fieldRA > 70.600000 and fieldDEC < -66.260000 and fieldDEC > -73.260000   for columns ['skyBrightness', 'filter', 'seeingFwhmEff', 'airmass', 'night', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA', 'observationId', 'fieldId']
Found 1284 visits
Running:  ['lmc']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting figures with "fieldRA < 91.200000 and fieldRA > 70.600000 and fieldDEC < -66.260000 and fieldDEC > -73.260000  " constraint now.
Plotting complete.


/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_maf/2.13.0.sims-93-g8bc2eb33+a14e41332a/python/lsst/sims/maf/db/resultsDb.py:279: UserWarning: Warning! Cannot save non-conforming summary statistic.
  warnings.warn('Warning! Cannot save non-conforming summary statistic.')


In [140]:
print(bundles[GName].dbCols)

{'skyBrightness', 'filter', 'seeingFwhmEff', 'airmass', 'night', 'fieldId', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA', 'observationId', 'fieldDec'}


In [141]:
print(len(bundles[GName].metricValues[0]))

1284


# Output

In [142]:
version="extr_v1"

In [143]:
#pdfilename="cadence_"+GName+"_"+dbtag+".csv"
pdfilename="cadence_"+ GName + "_" + dbtag + "_" + version + "_test.csv"

In [144]:
pdfilename

'cadence_lmc_baseline_2snaps_v1_extr_v1_test.csv'

In [145]:
df  = pd.DataFrame()

In [146]:
N=len(dbfields)

In [147]:
for index in np.arange(N):
    fieldname=dbfields[index]
    data=bundles[GName].metricValues[0][fieldname]
    df[fieldname]=data

In [148]:
df

,observationId,night,observationStartMJD,fiveSigmaDepth,filter,airmass,skyBrightness,seeingFwhmEff,fieldId,fieldRA,fieldDec
0,1923,3,59856.353356,24.032268,r,1.281545,21.065518,1.022068,-1,75.645541,-68.796842
1,1996,3,59856.387608,23.535707,i,1.242388,19.602172,0.770412,-1,77.676780,-66.549905
2,2774,4,59857.345243,23.968351,r,1.370114,21.009206,1.049511,-1,81.703694,-72.723577
3,2775,4,59857.345698,24.052231,r,1.275902,21.069316,1.005596,-1,71.421939,-68.581392
4,5117,7,59860.324066,22.102594,y,1.281857,18.145887,0.814007,-1,71.863300,-68.626264
...,...,...,...,...,...,...,...,...,...,...,...
1279,1911640,3456,63278.064273,23.659384,r,1.350042,20.930084,1.355133,-1,90.005355,-72.224572
1280,1912375,3457,63279.048438,23.359319,i,1.264468,20.140265,1.136952,-1,89.004168,-67.889378
1281,1912376,3457,63279.048885,23.208395,i,1.275802,20.131322,1.300346,-1,81.651530,-67.983175
1282,1912408,3457,63279.064749,22.829058,z,1.271342,19.190921,1.029019,-1,89.004168,-67.889378


In [149]:
df.to_csv(pdfilename)

In [150]:
!ls -l -t

total 2680
-rw-r--r--  1 dagoret  staff  193420 Apr  1 21:20 cadence_lmc_baseline_2snaps_v1_extr_v1_test.csv
drwxr-xr-x  5 dagoret  staff     160 Apr  1 21:20 output_galaxycadence
-rw-r--r--@ 1 dagoret  staff   38281 Apr  1 21:19 ExtractLMCCadence.ipynb
-rw-r--r--  1 dagoret  staff  197231 Apr  1 21:19 cadence_lmc_baseline_v1_extr_v1_test.csv
-rw-r--r--@ 1 dagoret  staff   39219 Apr  1 21:18 DumpLMCCadence.ipynb
-rw-r--r--  1 dagoret  staff  197231 Apr  1 21:18 cadence_lmc_baseline_v1_extr_v1.csv
-rw-r--r--  1 dagoret  staff  103980 Apr  1 21:13 cadence_lmc_baseline2018a_extr_v1_test.csv
-rw-r--r--  1 dagoret  staff  103980 Apr  1 21:12 cadence_lmc_baseline2018a_extr_v1.csv
-rw-r--r--@ 1 dagoret  staff  132955 Apr  1 20:56 cadence_lmc_baseline_2snaps_v1.csv
-rw-r--r--@ 1 dagoret  staff  134755 Jan 21 22:09 cadence_lmc_baseline_v1.csv
-rw-r--r--@ 1 dagoret  staff  102924 Jan 21 22:08 cadence_lmc_baseline2018a.csv
-rw-r--r--@ 1 dagoret  staff   17009 Jan 21 22:05 cadence_smc_baseline_v1.